# + Requirements

1. List of strategies on AllocateSmartly
2. CSVs of each strategy
3. Update every month (at rebalance date)
4. Import data table for each strategy on PortfolioVisualizer as "shares" and benchmarks
5. Update strategy (strategy of several strategies)
6.

In [1]:
import os
from selenium import webdriver
import pandas as pd
import datetime, time, csv

REMEMBER

export ASNAME='scubamut@gmail.com'

export ASPASSWD=''

export PVNAME='scubamut@gmail.com'

export PVPASSWD=''

In [2]:
# DON'T UPLOAD PASSWORDS TO GITHUB

os.environ.update({'PVNAME':'scubamut@gmail.com'})
os.environ.update({'PVPASSWD':'9D@3G!#qH5ZK*k^X*L%x'})
os.environ.update({'ASNAME':'scubamut@gmail.com'})
os.environ.update({'ASPASSWD':'LoZu#WJdjUvu3uFzLOst'})

# Allocate Smartly

In [3]:
# prepare Data folders

import shutil

try:
    shutil.rmtree(b'/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data')
except:
    pass

os.makedirs(b'/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data')
os.makedirs(b'/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/strategy_returns')
os.makedirs(b'/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/benchmark_returns')

## Login to allocatesmartly.com and generate CSV files for the Strategies


In [3]:
data_path = '/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/'

# browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
browser = webdriver.Chrome("/usr/local/share/chromedriver")
browser.get('https://allocatesmartly.com/login')
browser.set_window_position(1,1)
browser.maximize_window()
time.sleep(5)
user = browser.find_element_by_id('user_login')
user.send_keys(os.environ.get('ASNAME'))
password = browser.find_element_by_id('user_pass')
password.send_keys(os.environ.get('ASPASSWD'))
time.sleep(5)
login = browser.find_element_by_id('wp-submit')
login.click()

browser.get('https://allocatesmartly.com/members/strategies/')

strategies = pd.read_html(browser.page_source, header=0)[0]
strategies = strategies.filter(items=[c for c in strategies.columns][1:-2])
# remove Strategy 'Trading ...'
strategies.Strategy = strategies.Strategy.apply(lambda x: x[:x.find('Trading')])
strategies[38:]

,Strategy,AnnualReturn (20Y),SharpeRatio (20Y)
38,Philosophical Economics' Growth-Trend Timing,8.6%,0.61
39,Glenn's Paired Switching Strategy,9.3%,0.61
40,Novell's Tactical Bond Strategy,5.4%,0.60
41,Davis' Three Way Model,7.8%,0.55
42,60/40 Benchmark,5.8%,0.48
43,Faber's Sector Relative Strength (Sector RS),6.8%,0.46
44,Faber's Ivy Portfolio,6.7%,0.46


In [5]:
# for each strategy, get monthly table and generate CSV files 

for s in strategies.Strategy:
    print(s)
    browser.find_elements_by_link_text(s)[0].click()
    browser.find_element_by_id('sx-periodic-table')
    table = pd.read_html(browser.page_source, header=1)[2]
    table.columns = ['Year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Total']
    # need to change 60/40 to 60_40 for 
    table.to_csv(data_path + 'strategy_returns/' + s.replace('/','_')+'.csv')
    browser.execute_script("window.history.go(-1)")
    time.sleep(2)
    
browser.close()

Adaptive Asset Allocation
Vigilant Asset Allocation - Aggressive
Allocate Smartly's Meta Strategy
Protective Asset Allocation
Accelerating Dual Momentum
Defensive Asset Allocation
Protective Asset Allocation - CPR
Varadi's Percentile Channels
Keuning's Generalized Protective Momentum
Flexible Asset Allocation
US Risk Parity Trend Following
Efficiente Index
Stoken's Active Combined Asset (ACA)
Elastic Asset Allocation - Defensive
Elastic Asset Allocation - Offensive
Stoken's Active Combined Asset (ACA) - Monthly
Robust Asset Allocation - Balanced
Classical Asset Allocation - Defensive
US Max Diversification
Faber's Global Tactical Asset Alloc. - Agg. 6
Varadi's Minimum Correlation Portfolio
Faber's Trinity Portfolio Lite
Global Risk Parity Trend Following
Vigilant Asset Allocation - Balanced
Faber's Global Tactical Asset Alloc. 13 (GTAA 13)
Faber's Global Tactical Asset Alloc. 5 (GTAA 5)
Composite Dual Momentum
US Equal Risk Contribution
US Min Correlation
Faber's Global Tactical Asset 

In [6]:
# 
strats = strategies.copy()
strats.to_csv(data_path + 'strats.csv')

## Generate symbols (benchmark names) for use by Portfolio Visualizer

In [7]:
data_path = '/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/'
strategies = pd.read_csv(data_path + 'strats.csv')

In [8]:
#  new Stratnames column and remove brackets
strategies['Stratnames'] = pd.Series(strategies.Strategy).str.replace(r"\(.*\)","")
# rename 60/40 Benchmark to B6040 Benchmark
strategies['Stratnames'] = strategies.Stratnames.str.replace('40', 'B6040')
# remove developer from name
strategies['Stratnames'] = strategies.Stratnames.str.replace("/('\w+)|(\w+'\w+)|(\w+')|(\w+)/", "")
# new column for Developers
strategies['Developer'] = strategies.Strategy.str.extract("/('\w+)|(\w+'\w+)|(\w+')|(\w+)/", expand=True)[1]
# create unique name >= 6 characters for Portfolio Visualizer Bencmarks
strategies['Name'] = [''.join([c for c in s if c.isupper()]) for s in strategies.Stratnames]
strategies['Numbers'] = strategies.Stratnames.str.extract('(\d+)', expand=True).fillna('')
strategies['Symbol'] = strategies['Name'] + strategies['Numbers']
# # append 0s so that Share Name is at least 6 characters
strategies['Symbol'] = [s+'00000'[:6-len(s)]for s in strategies.Symbol]
strategies['Symbol'] = strategies['Symbol'].str.replace('4000', 'B6040')
strategies = strategies.filter(items=['Strategy', 'Stratnames', 'Symbol', 'AnnualReturn (20Y)', 'SharpeRatio (20Y)'])
strategies['Strategy'] = strategies['Strategy'].str.replace('/','_')
strategies[38:]

,Strategy,Stratnames,Symbol,AnnualReturn (20Y),SharpeRatio (20Y)
38,Philosophical Economics' Growth-Trend Timing,Philosophical Growth-Trend Timing,PGTT00,8.6%,0.61
39,Glenn's Paired Switching Strategy,Paired Switching Strategy,PSS000,9.3%,0.61
40,Novell's Tactical Bond Strategy,Tactical Bond Strategy,TBS000,5.4%,0.60
41,Davis' Three Way Model,Three Way Model,TWM000,7.8%,0.55
42,60_40 Benchmark,B6040 Benchmark,BB6040,5.8%,0.48
43,Faber's Sector Relative Strength (Sector RS),Sector Relative Strength,SRS000,6.8%,0.46
44,Faber's Ivy Portfolio,Ivy Portfolio,IP0000,6.7%,0.46


In [9]:
# verfy that all Securities are unique

len(strategies) == len(strategies.Symbol.unique())

True

In [10]:
strategies.to_csv('/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/ASstrategies.csv', index=None)
strategies[:5]

,Strategy,Stratnames,Symbol,AnnualReturn (20Y),SharpeRatio (20Y)
0,Adaptive Asset Allocation,Adaptive Asset Allocation,AAA000,12.2%,1.07
1,Vigilant Asset Allocation - Aggressive,Vigilant Asset Allocation - Aggressive,VAAA00,14.3%,1.05
2,Allocate Smartly's Meta Strategy,Allocate Meta Strategy,AMS000,8.6%,1.02
3,Protective Asset Allocation,Protective Asset Allocation,PAA000,9.6%,1.01
4,Accelerating Dual Momentum,Accelerating Dual Momentum,ADM000,15.1%,0.98


## Generate Benchmark Returns for use by Portfolio Visualizer 

In [11]:
def update_strategy_data(data_path, strategy, symbol):
    """
    For each strategy, save latest data table from website, 
    generate csv data to save benchmark with format suitable for
    PortfolioVisualizer
    
    - data_path : windows folder name for data
    - symbol : strategy Symbol 
    - strategy : strategy Name
    
    """
    df = pd.read_csv(data_path + 'strategy_returns/' + strategy + '.csv',index_col=[0] ).fillna('0.0%')

    df1 = pd.DataFrame(columns=['Period','Return'])
    df1 = df1.append({'Period': 'Period', 'Return': 'Return'}, ignore_index=True)
    for row in range(0,len(df)):
        for column in range(1,13):
            year, month, value = df.iloc[row,0], column, df.iloc[row, column]
#             period = (datetime.date (year, month, 1) - datetime.timedelta (days = 1)).strftime('%#m/%#d/%Y')
            next_month = datetime.date (year, month, 1).replace(day=28) + datetime.timedelta(days=4)  # this will never fail
            period = next_month - datetime.timedelta(days=next_month.day)
            newline = str(period) + u',' + value
            df1 = df1.append({'Period': period, 'Return': value}, ignore_index=True)

    df1.to_csv(data_path + 'benchmark_returns/' + symbol.replace('/','_')+'.csv', index=False, header=False, quoting=csv.QUOTE_NONNUMERIC) 

In [12]:
data_path = '/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/'

for n in range(len(strategies)):
# for n in [1]:
    symbol = strategies.Symbol[n]
    strategy = strategies.Strategy[n]
    print((symbol, strategy))
    update_strategy_data(data_path, strategy, symbol)
    
#browser.close()

('AAA000', 'Adaptive Asset Allocation')
('VAAA00', 'Vigilant Asset Allocation - Aggressive')
('AMS000', "Allocate Smartly's Meta Strategy")
('PAA000', 'Protective Asset Allocation')
('ADM000', 'Accelerating Dual Momentum')
('DAA000', 'Defensive Asset Allocation')
('PAACPR', 'Protective Asset Allocation - CPR')
('PC0000', "Varadi's Percentile Channels")
('GPM000', "Keuning's Generalized Protective Momentum")
('FAA000', 'Flexible Asset Allocation')
('USRPTF', 'US Risk Parity Trend Following')
('EI0000', 'Efficiente Index')
('ACA000', "Stoken's Active Combined Asset (ACA)")
('EAAD00', 'Elastic Asset Allocation - Defensive')
('EAAO00', 'Elastic Asset Allocation - Offensive')
('ACAM00', "Stoken's Active Combined Asset (ACA) - Monthly")
('RAAB00', 'Robust Asset Allocation - Balanced')
('CAAD00', 'Classical Asset Allocation - Defensive')
('USMD00', 'US Max Diversification')
('GTAAA6', "Faber's Global Tactical Asset Alloc. - Agg. 6")
('MCP000', "Varadi's Minimum Correlation Portfolio")
('TPL00

# Portfolio Visualizer

## Login to PortfolioVisualizer

In [27]:
from selenium import webdriver
import time

# browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
browser = webdriver.Chrome("/usr/local/share/chromedriver")
# browser = webdriver.Firefox()
browser.set_window_position(1,1)
# browser.set_window_size(1038, 875)
browser.maximize_window()
browser.get('https://www.portfoliovisualizer.com/login')
time.sleep(2)
user = browser.find_element_by_id('username')
user.send_keys(os.environ.get('PVNAME'))
password = browser.find_element_by_id('password')
password.send_keys(os.environ.get('PVPASSWD'))
time.sleep(2)
login = browser.find_element_by_id('submitButton')
login.click()

## Navigate to Import Benchmarks

In [28]:
len(strategies)

45

In [29]:
strategies[-3:]

,Strategy,Stratnames,Symbol,AnnualReturn (20Y),SharpeRatio (20Y)
42,60_40 Benchmark,B6040 Benchmark,BB6040,5.8%,0.48
43,Faber's Sector Relative Strength (Sector RS),Sector Relative Strength,SRS000,6.8%,0.46
44,Faber's Ivy Portfolio,Ivy Portfolio,IP0000,6.7%,0.46


In [30]:
data_path = '/home/scubamut/MEGAsync/WORK_IN_PROGRESS/pvautomate/Data/benchmark_returns/'

# browser.get('https://www.portfoliovisualizer.com/preferences#import')
browser.get('https://www.portfoliovisualizer.com/manage-benchmarks#import')

In [31]:
for n in range(len(strategies)):
# for n in [1]:
    
    symbol = strategies.Symbol[n]
    strategy = strategies.Strategy[n]
    print (symbol, strategy)
    
    browser.maximize_window()
    
    # Series Name
    browser.find_element_by_id("benchmarkName").clear()
    browser.find_element_by_id("benchmarkName").send_keys(strategy)

    # Upload Data File
    browser.find_element_by_id("upload").clear()
    browser.find_element_by_id("upload").send_keys(data_path + symbol +'.csv')
    
    # Series (Type)
    browser.find_element_by_id('seriesType_chosen').click()

    # choose SeriesType : li[1|2|3|4] eg li[1] gives Monthly Returns
    browser.find_element_by_xpath('//*[@id="seriesType_chosen"]/div/ul/li[1]').click()    

    # Percentage Values (2=Yes)
    browser.find_element_by_id('percentageValues_chosen').click()
    browser.find_element_by_xpath('//*[@id="percentageValues_chosen"]/div/ul/li[2]').click()

    # Assigned Ticker
    browser.find_element_by_id("benchmarkSymbol").clear()
    browser.find_element_by_id("benchmarkSymbol").send_keys(symbol)
    browser.find_element_by_id('benchmarkAssetClass_chosen').click()
    
    # NOTE: there must be 10 years of monthly data to assign as an Asset Class (2=Yes)
    # But don't need this for creating a benckmark

    # Asset Class (2=Yes)
    browser.find_element_by_css_selector('#benchmarkAssetClass_chosen > div > ul > li:nth-child(2)').click()

    # Import Data Series
    browser.implicitly_wait(90)
    browser.find_element_by_id("importBenchmarkButton").click()
    
    print(browser.find_element_by_class_name("alert").text)

    browser.execute_script("window.history.go(-1)")
    time.sleep(2)
    
browser.close()    

AAA000 Adaptive Asset Allocation
Benchmark series 'Adaptive Asset Allocation' imported.
VAAA00 Vigilant Asset Allocation - Aggressive
Benchmark series 'Vigilant Asset Allocation - Aggressive' imported.
AMS000 Allocate Smartly's Meta Strategy
Benchmark series 'Allocate Smartly's Meta Strategy' imported.
PAA000 Protective Asset Allocation
Benchmark series 'Protective Asset Allocation' imported.
ADM000 Accelerating Dual Momentum
Benchmark series 'Accelerating Dual Momentum' imported.
DAA000 Defensive Asset Allocation
Benchmark series 'Defensive Asset Allocation' imported.
PAACPR Protective Asset Allocation - CPR
Benchmark series 'Protective Asset Allocation - CPR' imported.
PC0000 Varadi's Percentile Channels
Benchmark series 'Varadi's Percentile Channels' imported.
GPM000 Keuning's Generalized Protective Momentum
Benchmark series 'Keuning's Generalized Protective Momentum' imported.
FAA000 Flexible Asset Allocation
Benchmark series 'Flexible Asset Allocation' imported.
USRPTF US Risk Par

In [ ]:
# REMEMBER THAT THE LAST MONTH MAY BE INCOMPLETE!!

# SCRATCH